In [3]:
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"import.sc"))
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"a.scala"))


Compiling /home/jovyan/data/(console)
Compiling /home/jovyan/data/(console)


In [4]:
import Diss._
import UaMeta._

import Diss._
import UaMeta._

In [5]:
  lazy val badFiles: Set[(String, String)] =
    scala.io.Source.fromFile("all.ban", "UTF-8").getLines.map { line =>
      val split = line.split("\t")
      split.head -> split(1)
    }.toSet


  def extractNumSimple3(str: String): List[String] = {
    if (str.count(_ == 177.toChar) <= 4) {
      val main: String = str.replaceAll("[.,]", "").map(c => if (c.isDigit) c else ' ')
      val list = main.split(" ").filter(_.nonEmpty)
            val filter: Array[String] = list.filter { n =>
        n match {
          case _ if n.startsWith("0") => true
          case _ if !n.startsWith("0") && n.length <= 4 && n.toInt <= 200 => false
          case _ if !n.startsWith("0") && n.length <= 4 && n.toInt >= 1900 && n.toInt <= 2024 => false
          case _ => true
        }
      }

      filter.sliding(3, 1).map(t => t.mkString("/")).toList
    } else {
      Nil
    }
  }

  def uaFileHandleToNum(work: String): List[(List[String], Int)] = {
    import scala.util.Try
    val numbers: List[(List[String], Int)] = Try {

      val a = work.split(12.toChar)

      val zip: Array[(String, Int)] = a.zipWithIndex

      val sub: List[(String, Int)] = zip.toList.sliding(3).toList.flatMap { array =>
        if (array(1)._1.toUpperCase.contains("ТАБЛИЦ")) {
          array
        } else {
          List.empty
        }
      }.distinct.sortBy(_._2)

      val res: List[(List[String], Int)] = sub.map(x => (extractNumSimple3(x._1), x._2))

      res.filter(_._1.size >= 8)
    }.getOrElse(List())

    numbers
  }

  def getDirPath(id: String): String = {
    s"txt/${id.takeRight(3).toInt}/$id/"
  }

  def readById(id: String): String = {
    val dir = getDirPath(id)
    val all: Vector[String] = FileUtils.traverse(dir).filter(_.endsWith(".txt")).filterNot(x => isAutoRefUA(x)).filterNot(x => badFiles.contains(id -> x.split("/").last)).map(read)
    all.mkString(12.toChar.toString)
  }

  def uaReadBy1000(id: Int) = {
    val iter: Iterator[String] = uaMetaVec.filter(x => x.id.takeRight(3).toInt == id).iterator.flatMap { meta =>
      val work = readById(meta.id)

      val tables: List[(List[String], Int)] = uaFileHandleToNum(work)

      val numUaId: Int = idUaRevMap(meta.id)

      tables.flatMap { case (pageNumbers, page) =>
        pageNumbers.map { pmNumber =>
          s"$pmNumber\tU${numUaId}:${page}"
        }
      }
    }

    FileUtils.write(s"uaPM$id.st", iter)
  }


badFiles: Set[(String, String)] = [lazy]
defined function extractNumSimple3
defined function uaFileHandleToNum
defined function getDirPath
defined function readById
defined function uaReadBy1000

In [6]:
uaReadBy1000(0)

In [7]:
(1 to 999).toVector.par.foreach(x => uaReadBy1000(x))

In [8]:
  def rufileHandleToNum(fileName: String): List[(List[String], Int)] = {
    val bibList = List[String](
      "СПИСОК ЛИТЕРАТУРЫ", "СПИСОК ИСПОЛЬЗОВАННОЙ", "СПИСОК ИСПОЛЬЗУЕМОЙ", "СПИСОКЛИТЕРАТУРЫ", "УКАЗАТЕЛЬ ЛИТЕРАТУРЫ",
      "СПИСОК ЦИТИРУЕМОЙ", "ИСПОЛЬЗОВАННАЯ ЛИТЕРАТУРА", "БИБЛИОГРАФИЧЕСКИЙ СПИСОК", "СПИСОК ИСПОЛЬЗОВАННЫХ",
      "БИБЛИОГРАФИЧЕСКИЙ", "БИБЛИОГРАФИЯ", "СПИСОК ИСТОЧНИКОВ", "ЛИТЕРАТУРА")

    val endList = List[String]("ВЫВОДЫ", "ЗАКЛЮЧЕНИЕ")

    import scala.util.Try
    val numbers: List[(List[String], Int)] = Try {

      val a: Array[String] = {
        import java.io._
        val br = new BufferedReader(new InputStreamReader(
          new FileInputStream(fileName), "UTF-8"))
        val res = br.lines().toArray.toList.map(_.toString).mkString(" ").split(12.toChar)
        br.close()
        res
      }

      val zip: Array[(String, Int)] = a.zipWithIndex
      val rev = zip.reverse

      val bib: Option[Int] = rev.take((rev.size * 3 / 4).toInt).find { case (str, _) =>
        val list = str.take(100).toUpperCase.filter(x => x.isLetter || x == ' ').split(" ").filter(_.nonEmpty)
        val head = list.mkString(" ")
        bibList.exists(prefix => head.startsWith(prefix))
      }.map(_._2)

      val end: Option[Int] = rev.take((rev.size * 3 / 4).toInt).find { case (str, _) =>
        val list = str.take(100).toUpperCase.filter(x => x.isLetter || x == ' ').split(" ").filter(_.nonEmpty)
        val head = list.mkString(" ")
        endList.exists(prefix => head.startsWith(prefix))
      }.map(_._2)

      val endPos = (bib, end) match {
        case (Some(b), Some(e)) => scala.math.min(b, e)
        case (None, Some(e)) => e
        case (Some(b), None) => b
        case _ => a.size - 10
      }

      val sub: List[(String, Int)] = zip.toList.slice(30, endPos).sliding(3).toList.flatMap { array =>
        if (array(1)._1.toUpperCase.contains("ТАБЛИЦ")) {
          array
        } else {
          List.empty
        }
      }.distinct.sortBy(_._2)

      val res: List[(List[String], Int)] = sub.map(x => (extractNumSimple3(x._1), x._2))

      res.filter(_._1.size >= 10)
    }.getOrElse(List())

    numbers
  }


  def ruDirHandle(id: Int): Unit = {
    val iter = FileUtils.traverse(s"data/d/$id").iterator.flatMap { fileName =>
      val i = fileName.split("/").last
      val rgbId: Int = all(i)

      val tables: List[(List[String], Int)] = rufileHandleToNum(fileName)

      tables.flatMap { case (pageNumbers, page) =>
        pageNumbers.map { pmNumber =>
          s"$pmNumber\tD${rgbId}:${page}"
        }
      }
    }

    FileUtils.write(s"rust$id.st", iter)
  }


defined function rufileHandleToNum
defined function ruDirHandle

In [9]:
ruDirHandle(0)

In [10]:
(1 to 999).toVector.par.foreach(x => ruDirHandle(x))

In [11]:
  val iter = GroupIterator[String, String, Vector[String]](scala.io.Source.fromFile("st.all.sort.st").getLines, x => {
    val split = x.split("\t")
    split.head -> Vector(split.last)
  }, _ ++ _).filter(x => x._2.size >= 2 && x._2.size <= 50).map {
    case (head, vector) => s"$head\t${vector.mkString("\t")}"
  }

  FileUtils.write("st.all.grouped.st", iter)


iter: Iterator[String] = empty iterator

In [12]:
    val iter = scala.io.Source.fromFile("st.all.sort.st").getLines.map { line =>
      line.split("\t").reverse.mkString("\t")
    }

    FileUtils.write("st.all.sort.rev.st", iter)


iter: Iterator[String] = empty iterator

In [ ]:
  case class PMTableSource(source: String) {
    def isRuDiss: Boolean = source.head == 'D'

    def isUaDiss: Boolean = source.head == 'U'

    def id: Int = source.split(":").head.drop(1).toInt

    def page: Int = source.split(":").last.toInt

    def year: Int = {
      if (isUaDiss) {
        getUaMetaById(id).year
      } else {
        scala.util.Try {
          getDissMeta(id).year.toInt
        }.getOrElse(2023)
      }
    }

    def ruMeta: MetaData2 = getDissMeta(id)

    def uaMeta: UaMeta = getUaMetaById(id)
  }

  case class PMTable(source: PMTableSource, numbers: Vector[String]) {}

  lazy val inverseIndex: Map[String, Vector[PMTableSource]] = scala.io.Source.fromFile("st.all.grouped.st").getLines.map { line =>
    val split = line.split("\t")
    split.head -> split.drop(1).toVector.map(PMTableSource.apply)
  }.toMap.withDefaultValue(Vector.empty)


  lazy val pmTables: Vector[PMTable] = GroupIterator[String, String, Vector[String]](scala.io.Source.fromFile("st.all.byWork.st").getLines, x => {
    val split = x.split("\t")
    split.head -> Vector(split.last)
  }, _ ++ _).map { case (source, numbers) => PMTable(PMTableSource(source), numbers)
  }.toVector

  lazy val pmTablesMap: Map[PMTableSource, PMTable] = pmTables.map { table =>
    table.source -> table
  }.toMap


In [ ]:
  def findCandidates(table: PMTable): Vector[PMTable] = {
    val mainYear = table.source.year
    val sources: Vector[PMTableSource] = table.numbers.flatMap(number => inverseIndex(number)).filter(_.isUaDiss).filter(x => mainYear >= x.year)

    val options: Vector[PMTableSource] = sources.groupBy(identity).mapValues(_.size).filter(_._2 >= 6).toVector.take(5).map(_._1)

    options.flatMap { opt =>
      pmTablesMap.get(opt)
    }
  }


  def splitPair(value: String): (String, String, String) = {
    val split = value.split("/")
    (split.head, split(1), split.last)
  }


  def comparePair(fst: String, snd: String): Boolean = {
    val (fa, fb, fc) = splitPair(fst)
    val (sa, sb, sc) = splitPair(snd)


    val lev1 = editDist(fa, sa)
    val lev2 = editDist(fb, sb)
    val lev3 = editDist(fc, sc)


    (fst, snd, lev1, lev2, lev3) match {
      case (f, s, l1, l2, l3) if l1 + l2 + l3 <= 1 => true
      case _ => false
    }
  }

  def similarity(list: Seq[(Int, Int)]): Double = {
    val s = list.sortBy(_._1).toVector
    var res = 0
    for {i <- s.indices
         j <- i + 1 until s.size
         if s(i)._2 > s(j)._2
         } {
      res = res + 1
    }

    1 - (res.toDouble / (s.size * s.size))
  }

  def similarityLinear(list: Seq[(Int, Int)]): Double = {
    val s = list.sortBy(_._1).toVector
    var res = 0
    for {i <- 0 until s.size - 2
         if s(i)._2 > s(i + 1)._2
         } {
      res = res + 1
    }

    1 - (res.toDouble / (s.size - 1))
  }


  def compareTwoTables(fst: Vector[String], snd: Vector[String]): (Int, Int, Int, Int, Double, Double, Int, Int, Int, Int) = {
    val fstU = fst.zipWithIndex
    val sndSet = snd.toSet
    val sndZip = snd.zipWithIndex

    val fuzzy: Vector[(Int, Int)] = fstU.filter(x => !sndSet.contains(x._1)).flatMap {
      f => sndZip.find(s => comparePair(f._1, s._1)).map(s => (f._2, s._2))
    }

    val eq: Vector[(Int, Int)] = fstU.filter { case (f, idx) =>
      sndSet.contains(f)
    }.flatMap(f => sndZip.find(x => f._1 == x._1).map(x => (f._2, x._2)))

    val tuples: Vector[(Int, Int)] = fuzzy ++ eq

    val f1 = tuples.map(p => fst(p._1)).distinct.size
    val s1 = tuples.map(p => snd(p._2)).distinct.size

    (fuzzy.size, eq.size, fst.size, snd.size, similarity(tuples), similarityLinear(tuples), fst.distinct.size, snd.distinct.size, f1, s1)
  }

  def validate(fst: PMTable, snd: PMTable): Option[(Int, Int, Int, Int, Double, Double, Int, Int, Int, Int, PMTable, PMTable)] = {
    val s2 = compareTwoTables(fst.numbers, snd.numbers)
    if (s2._1 + s2._2 >= 15 && (s2._1 + s2._2 + 0.1) / s2._3 >= 0.5 && s2._5 >= 0.95) {
      Some(s2._1, s2._2, s2._3, s2._4, s2._5, s2._6, s2._7, s2._8, s2._9, s2._10, fst, snd)
    } else {
      None
    }
  }


In [ ]:
  val pmIter: Iterator[String] = pmTables.filter(_.source.isRuDiss).sortBy(x => x.source.year)(Ordering[Int].reverse).iterator.flatMap { table =>
    val candidates: Vector[PMTable] = findCandidates(table)
    val options: Vector[(Int, Int, Int, Int, Double, Double, Int, Int, Int, Int, PMTable, PMTable)] = candidates.flatMap(x => validate(table, x))

    if (options.nonEmpty) {
      val mainMeta = table.source.ruMeta

      options.flatMap { case (f, eq, total1, total2, sim, simL, fstUniq, sndUniq, f1, s1, t1, t2) =>
        val otherMeta = t2.source.uaMeta
        val header = List[String](mainMeta.id, mainMeta.year, mainMeta.toString, otherMeta.id, otherMeta.year.toString, otherMeta.toString)
        val list = List[String]("", "", "", "", "", "", "page: " + (t1.source.page + 1).toString, "page: " + (t2.source.page + 1).toString,
                            "fuzzy: " + f.toString, "eq: " + eq.toString, "total1: " + total1.toString, "total2: " + total2.toString, "sim: " + sim.toString, "simL:" + simL.toString, "fstUniq: " + fstUniq.toString, "sndUniq: " + sndUniq.toString,
                            "dSize1: " + f1.toString, "dSize2: " + s1.toString)
        Vector(header.mkString("\t"), list.mkString("\t"))
      }
    } else {
      Vector.empty
    }
  }

  FileUtils.write("table.simple.UA2RU.csv", pmIter)



In [ ]:
  def findCandidatesRu(table: PMTable): Vector[PMTable] = {
    val mainYear = table.source.year
    val sources: Vector[PMTableSource] = table.numbers.flatMap(number => inverseIndex(number)).filter(_.isRuDiss).filter(x => mainYear >= x.year)

    val options: Vector[PMTableSource] = sources.groupBy(identity).mapValues(_.size).filter(_._2 >= 6).toVector.take(5).map(_._1)

    options.flatMap { opt =>
      pmTablesMap.get(opt)
    }
  }


In [ ]:
    val pmIter: Iterator[String] = pmTables.filter(_.source.isUaDiss).sortBy(x => x.source.year)(Ordering[Int].reverse).iterator.flatMap { table =>
      val candidates: Vector[PMTable] = findCandidatesRu(table)
      val options: Vector[(Int, Int, Int, Int, Double, Double, Int, Int, Int, Int, PMTable, PMTable)] = candidates.flatMap(x => validate(table, x))

      if (options.nonEmpty) {
        val mainMeta = table.source.uaMeta

        options.flatMap { case (f, eq, total1, total2, sim, simL, fstUniq, sndUniq, f1, s1, t1, t2) =>
          val otherMeta = t2.source.ruMeta
          val header = List[String](mainMeta.id, mainMeta.year.toString, mainMeta.toString, otherMeta.id, otherMeta.year.toString, otherMeta.toString)
          val list = List[String]("", "", "", "", "", "", "page: " + (t1.source.page + 1).toString, "page: " + (t2.source.page + 1).toString,
            "fuzzy: " + f.toString, "eq: " + eq.toString, "total1: " + total1.toString, "total2: " + total2.toString, "sim: " + sim.toString, "simL:" + simL.toString, "fstUniq: " + fstUniq.toString, "sndUniq: " + sndUniq.toString,
            "dSize1: " + f1.toString, "dSize2: " + s1.toString)
          Vector(header.mkString("\t"), list.mkString("\t"))
        }
      } else {
        Vector.empty
      }
    }

    FileUtils.write("table.simple.RU2UA.csv", pmIter)


In [ ]:
  def findCandidatesUa(table: PMTable): Vector[PMTable] = {
    val mainYear = table.source.year
    val sources: Vector[PMTableSource] = table.numbers.flatMap(number => inverseIndex(number)).filter(_.isUaDiss).filter(x => mainYear >= x.year)

    val options: Vector[PMTableSource] = sources.groupBy(identity).mapValues(_.size).filter(_._2 >= 6).toVector.take(5).map(_._1)

    options.flatMap { opt =>
      pmTablesMap.get(opt)
    }
  }


In [ ]:
    val pmIter: Iterator[String] = pmTables.filter(_.source.isUaDiss).sortBy(x => x.source.year)(Ordering[Int].reverse).iterator.flatMap { table =>
      val candidates: Vector[PMTable] = findCandidatesUa(table)
      val options: Vector[(Int, Int, Int, Int, Double, Double, Int, Int, Int, Int, PMTable, PMTable)] = candidates.flatMap(x => validate(table, x)).filter(x => x._11.source != x._12.source)

      if (options.nonEmpty) {
        val mainMeta = table.source.uaMeta

        options.flatMap { case (f, eq, total1, total2, sim, simL, fstUniq, sndUniq, f1, s1, t1, t2) =>
          val otherMeta = t2.source.uaMeta
          val header = List[String](mainMeta.id, mainMeta.year.toString, mainMeta.toString, otherMeta.id, otherMeta.year.toString, otherMeta.toString)
          val list = List[String]("", "", "", "", "", "", "page: " + (t1.source.page + 1).toString, "page: " + (t2.source.page + 1).toString,
            "fuzzy: " + f.toString, "eq: " + eq.toString, "total1: " + total1.toString, "total2: " + total2.toString, "sim: " + sim.toString, "simL:" + simL.toString, "fstUniq: " + fstUniq.toString, "sndUniq: " + sndUniq.toString,
            "dSize1: " + f1.toString, "dSize2: " + s1.toString)
          Vector(header.mkString("\t"), list.mkString("\t"))
        }
      } else {
        Vector.empty
      }
    }

    FileUtils.write("table.simple.UA2UA.csv", pmIter)
